In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json
import math
import os

import cv2
from PIL import Image
import numpy as np
from keras import layers
from tensorflow.keras.applications import ResNet50,MobileNet, DenseNet201, InceptionV3, NASNetLarge, InceptionResNetV2, NASNetMobile
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
from tqdm import tqdm
import tensorflow as tf
from keras import backend as K
from functools import partial
from sklearn import metrics
from collections import Counter
import json
import itertools
import h5py


%matplotlib inline

In [3]:
def Dataset_loader(DIR, RESIZE, sigmaX=10):
    IMG = []
    read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))
    for IMAGE_NAME in tqdm(os.listdir(DIR)):
        PATH = os.path.join(DIR,IMAGE_NAME)
        _, ftype = os.path.splitext(PATH)
        if ftype == ".png" or ".tif":
            img = read(PATH)
           
            img = cv2.resize(img, (RESIZE,RESIZE))
           
            IMG.append(np.array(img))
    return IMG

In [40]:
label_benign = []
shape_benign = []
data_benign = []
types = os.listdir('/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB')
print(types)
for i in range(len(types)):
  cases_path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/'+types[i]
  print(cases_path)
  cases = os.listdir(cases_path)
  print(cases)
  for j in range(len(cases)):
    magnification_path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/'+types[i]+'/'+cases[j]
    print(magnification_path)
    magnifications = os.listdir(magnification_path)
    print(magnifications)
    for k in range(len(magnifications)): 
      path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/'+types[i]+'/'+cases[j]+'/'+magnifications[k]
      print(path)
      l = 'benign_'+types[i]+'_'+cases[j]+'_'+magnifications[k]
      print(l)
      label_benign.append(l)
      k = np.array(Dataset_loader(path,128))
      data_benign.append(k)
      shape_benign.append(k.shape)
print(label_benign)
print(shape_benign)
print(len(label_benign),len(shape_benign),len(data_benign))

['fibroadenoma', 'tubular_adenoma', 'adenosis', 'phyllodes_tumor']
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/fibroadenoma
['SOB_B_F_14-25197', 'SOB_B_F_14-23222AB', 'SOB_B_F_14-29960AB', 'SOB_B_F_14-9133', 'SOB_B_F_14-23060AB', 'SOB_B_F_14-23060CD', 'SOB_B_F_14-21998CD', 'SOB_B_F_14-21998EF', 'SOB_B_F_14-14134E', 'SOB_B_F_14-14134']
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/fibroadenoma/SOB_B_F_14-25197
['400X', '40X', '200X', '100X']
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/fibroadenoma/SOB_B_F_14-25197/400X
benign_fibroadenoma_SOB_B_F_14-25197_400X


100%|██████████| 33/33 [00:00<00:00, 47.49it/s]


/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/fibroadenoma/SOB_B_F_14-25197/40X
benign_fibroadenoma_SOB_B_F_14-25197_40X


100%|██████████| 22/22 [00:00<00:00, 45.62it/s]


/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/fibroadenoma/SOB_B_F_14-25197/200X
benign_fibroadenoma_SOB_B_F_14-25197_200X


...

/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/phyllodes_tumor/SOB_B_PT_14-22704/200X
benign_phyllodes_tumor_SOB_B_PT_14-22704_200X


100%|██████████| 42/42 [00:19<00:00,  2.19it/s]


/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/phyllodes_tumor/SOB_B_PT_14-22704/100X
benign_phyllodes_tumor_SOB_B_PT_14-22704_100X


100%|██████████| 39/39 [00:16<00:00,  2.32it/s]

['benign_fibroadenoma_SOB_B_F_14-25197_400X', 'benign_fibroadenoma_SOB_B_F_14-25197_40X', 'benign_fibroadenoma_SOB_B_F_14-25197_200X', 'benign_fibroadenoma_SOB_B_F_14-25197_100X', 'benign_fibroadenoma_SOB_B_F_14-23222AB_100X', 'benign_fibroadenoma_SOB_B_F_14-23222AB_40X', 'benign_fibroadenoma_SOB_B_F_14-23222AB_400X', 'benign_fibroadenoma_SOB_B_F_14-23222AB_200X', 'benign_fibroadenoma_SOB_B_F_14-29960AB_400X', 'benign_fibroadenoma_SOB_B_F_14-29960AB_40X', 'benign_fibroadenoma_SOB_B_F_14-29960AB_100X', 'benign_fibroadenoma_SOB_B_F_14-29960AB_200X', 'benign_fibroadenoma_SOB_B_F_14-9133_40X', 'benign_fibroadenoma_SOB_B_F_14-9133_400X', 'benign_fibroadenoma_SOB_B_F_14-9133_100X', 'benign_fibroadenoma_SOB_B_F_14-9133_200X', 'benign_fibroadenoma_SOB_B_F_14-23060AB_40X', 'benign_fibroadenoma_SOB_B_F_14-23060AB_400X', 'benign_fibroadenoma_SOB_B_F_14-23060AB_100X', 'benign_fibroadenoma_SOB_B_F_14-23060AB_200X', 'benign_fibroadenoma_SOB_B_F_14-23060CD_400X', 'benign_fibroadenoma_SOB_B_F_14-23060

In [41]:
label_malignant = []
shape_malignant = []
data_malignant = []
types = os.listdir('/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB')
print(types)
for i in range(len(types)):
  cases_path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/'+types[i]
  print(cases_path)
  cases = os.listdir(cases_path)
  print(cases)
  for j in range(len(cases)):
    magnification_path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/'+types[i]+'/'+cases[j]
    print(magnification_path)
    magnifications = os.listdir(magnification_path)
    print(magnifications)
    for k in range(len(magnifications)): 
      path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/'+types[i]+'/'+cases[j]+'/'+magnifications[k]
      print(path)
      l = 'malignant_'+types[i]+'_'+cases[j]+'_'+magnifications[k]
      print(l)
      label_malignant.append(l)
      k = np.array(Dataset_loader(path,128))
      data_malignant.append(k)
      shape_malignant.append(k.shape)
print(label_malignant)
print(shape_malignant)
print(len(label_malignant),len(shape_malignant),len(data_malignant))

['papillary_carcinoma', 'mucinous_carcinoma', 'ductal_carcinoma', 'lobular_carcinoma']
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/papillary_carcinoma
['SOB_M_PC_15-190EF', 'SOB_M_PC_14-9146', 'SOB_M_PC_14-15704', 'SOB_M_PC_14-12465', 'SOB_M_PC_14-19440', 'SOB_M_PC_14-15687B']
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/papillary_carcinoma/SOB_M_PC_15-190EF
['40X', '400X', '100X', '200X']
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/papillary_carcinoma/SOB_M_PC_15-190EF/40X
malignant_papillary_carcinoma_SOB_M_PC_15-190EF_40X


100%|██████████| 19/19 [00:08<00:00,  2.24it/s]


/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/papillary_carcinoma/SOB_M_PC_15-190EF/400X
malignant_papillary_carcinoma_SOB_M_PC_15-190EF_400X


100%|██████████| 15/15 [00:07<00:00,  2.01it/s]


/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/papillary_carcinoma/SOB_M_PC_15-190EF/100X
malignant_papillary_carcinoma_SOB_M_PC_15-190EF_100X


...

/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/lobular_carcinoma/SOB_M_LC_14-13412/200X
malignant_lobular_carcinoma_SOB_M_LC_14-13412_200X


100%|██████████| 32/32 [00:13<00:00,  2.43it/s]


/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/lobular_carcinoma/SOB_M_LC_14-13412/100X
malignant_lobular_carcinoma_SOB_M_LC_14-13412_100X


100%|██████████| 33/33 [00:10<00:00,  3.05it/s]

['malignant_papillary_carcinoma_SOB_M_PC_15-190EF_40X', 'malignant_papillary_carcinoma_SOB_M_PC_15-190EF_400X', 'malignant_papillary_carcinoma_SOB_M_PC_15-190EF_100X', 'malignant_papillary_carcinoma_SOB_M_PC_15-190EF_200X', 'malignant_papillary_carcinoma_SOB_M_PC_14-9146_40X', 'malignant_papillary_carcinoma_SOB_M_PC_14-9146_400X', 'malignant_papillary_carcinoma_SOB_M_PC_14-9146_100X', 'malignant_papillary_carcinoma_SOB_M_PC_14-9146_200X', 'malignant_papillary_carcinoma_SOB_M_PC_14-15704_40X', 'malignant_papillary_carcinoma_SOB_M_PC_14-15704_400X', 'malignant_papillary_carcinoma_SOB_M_PC_14-15704_200X', 'malignant_papillary_carcinoma_SOB_M_PC_14-15704_100X', 'malignant_papillary_carcinoma_SOB_M_PC_14-12465_40X', 'malignant_papillary_carcinoma_SOB_M_PC_14-12465_400X', 'malignant_papillary_carcinoma_SOB_M_PC_14-12465_100X', 'malignant_papillary_carcinoma_SOB_M_PC_14-12465_200X', 'malignant_papillary_carcinoma_SOB_M_PC_14-19440_400X', 'malignant_papillary_carcinoma_SOB_M_PC_14-19440_40X', 

In [42]:
f1 = h5py.File("/content/drive/MyDrive/Project/BreakHis.hdf5", "w")

In [43]:
for i in range(len(label_benign)):
  dataset = f1.create_dataset(label_benign[i],shape_benign[i],dtype='i',data=data_benign[i])

In [44]:
for i in range(len(label_malignant)):
  dataset = f1.create_dataset(label_malignant[i],shape_malignant[i],dtype='i',data=data_malignant[i])

In [45]:
list(f1.keys())

['benign_adenosis_SOB_B_A_14-22549AB_100X',
 'benign_adenosis_SOB_B_A_14-22549AB_200X',
 'benign_adenosis_SOB_B_A_14-22549AB_400X',
 'benign_adenosis_SOB_B_A_14-22549AB_40X',
 'benign_adenosis_SOB_B_A_14-22549CD_100X',
 'benign_adenosis_SOB_B_A_14-22549CD_200X',
 'benign_adenosis_SOB_B_A_14-22549CD_400X',
 'benign_adenosis_SOB_B_A_14-22549CD_40X',
 'benign_adenosis_SOB_B_A_14-22549G_100X',
 'benign_adenosis_SOB_B_A_14-22549G_200X',
 'benign_adenosis_SOB_B_A_14-22549G_400X',
 'benign_adenosis_SOB_B_A_14-22549G_40X',
 'benign_adenosis_SOB_B_A_14-29960CD_100X',
 'benign_adenosis_SOB_B_A_14-29960CD_200X',
 'benign_adenosis_SOB_B_A_14-29960CD_400X',
 'benign_adenosis_SOB_B_A_14-29960CD_40X',
 'benign_fibroadenoma_SOB_B_F_14-14134E_100X',
 'benign_fibroadenoma_SOB_B_F_14-14134E_200X',
 'benign_fibroadenoma_SOB_B_F_14-14134E_400X',
 'benign_fibroadenoma_SOB_B_F_14-14134E_40X',
 'benign_fibroadenoma_SOB_B_F_14-14134_100X',
 'benign_fibroadenoma_SOB_B_F_14-14134_200X',
 'benign_fibroadenoma_SOB

In [46]:
f1.close()

In [47]:
f1 = h5py.File("/content/drive/MyDrive/Project/BreakHis.hdf5", "r")

In [48]:
list(f1.keys())

['benign_adenosis_SOB_B_A_14-22549AB_100X',
 'benign_adenosis_SOB_B_A_14-22549AB_200X',
 'benign_adenosis_SOB_B_A_14-22549AB_400X',
 'benign_adenosis_SOB_B_A_14-22549AB_40X',
 'benign_adenosis_SOB_B_A_14-22549CD_100X',
 'benign_adenosis_SOB_B_A_14-22549CD_200X',
 'benign_adenosis_SOB_B_A_14-22549CD_400X',
 'benign_adenosis_SOB_B_A_14-22549CD_40X',
 'benign_adenosis_SOB_B_A_14-22549G_100X',
 'benign_adenosis_SOB_B_A_14-22549G_200X',
 'benign_adenosis_SOB_B_A_14-22549G_400X',
 'benign_adenosis_SOB_B_A_14-22549G_40X',
 'benign_adenosis_SOB_B_A_14-29960CD_100X',
 'benign_adenosis_SOB_B_A_14-29960CD_200X',
 'benign_adenosis_SOB_B_A_14-29960CD_400X',
 'benign_adenosis_SOB_B_A_14-29960CD_40X',
 'benign_fibroadenoma_SOB_B_F_14-14134E_100X',
 'benign_fibroadenoma_SOB_B_F_14-14134E_200X',
 'benign_fibroadenoma_SOB_B_F_14-14134E_400X',
 'benign_fibroadenoma_SOB_B_F_14-14134E_40X',
 'benign_fibroadenoma_SOB_B_F_14-14134_100X',
 'benign_fibroadenoma_SOB_B_F_14-14134_200X',
 'benign_fibroadenoma_SOB

In [49]:
a = f1['benign_fibroadenoma_SOB_B_F_14-14134_100X']
print(len(a),a.shape)

31 (31, 128, 128, 3)


In [50]:
f1.close()